In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/adapter_model.safetensors
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/training_args.bin
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/adapter_config.json
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/README.md
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/tokenizer.json
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/tokenizer_config.json
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/chat_template.jinja
/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B/special_tokens_map.json
/kaggle/input/ultramedical-preference/ultramedical_preference.json


## Setup and Installation

In [2]:
# Install required packages
!pip install -q transformers datasets accelerate peft bitsandbytes trl torch ijson huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 10.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [4]:
import torch
import random
import json
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOConfig, DPOTrainer
import numpy as np

print("✅ All packages imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

2026-01-27 08:30:00.121962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769502600.317931      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769502600.373158      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769502600.876793      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769502600.876837      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769502600.876840      55 computation_placer.cc:177] computation placer alr

✅ All packages imported successfully!
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla P100-PCIE-16GB
GPU Memory: 17.06 GB


## Configuration

In [5]:
print("\n" + "="*80)
print("CONFIGURATION")
print("="*80)

# Model Configuration
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
MODEL_SFT_PATH = "/kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B"  
MODEL_D_PATH = "/kaggle/working/model_d_dpo"  # Output path

# Data Configuration
PREFERENCE_DATA_PATH = "/kaggle/input/ultramedical-preference/ultramedical_preference.json"  # Update: Your preference data

# DPO Training Configuration
DPO_CONFIG = {
    "output_dir": MODEL_D_PATH,
    "num_train_epochs": 1,
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 16,
    "learning_rate": 5e-6,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    "bf16": True,
    "logging_steps": 10,
    "save_steps": 100,
    "eval_strategy": "steps",
    "eval_steps": 100,
    "save_total_limit": 2,
    "gradient_checkpointing": True,
    "remove_unused_columns": False,
    "beta": 0.1,  # DPO temperature (higher = more conservative)
    "max_length": 1024,
    "max_prompt_length": 512,
    "beta":0.1,
}

# LoRA Configuration (if not already in Model_SFT)
LORA_CONFIG = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"]
}

# Quantization Configuration
BNB_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"Base Model: {BASE_MODEL}")
print(f"Model_SFT Path: {MODEL_SFT_PATH}")
print(f"Output Path: {MODEL_D_PATH}")
print(f"Preference Data: {PREFERENCE_DATA_PATH}")
print("✅ Configuration set!")


CONFIGURATION
Base Model: meta-llama/Llama-3.2-3B-Instruct
Model_SFT Path: /kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B
Output Path: /kaggle/working/model_d_dpo
Preference Data: /kaggle/input/ultramedical-preference/ultramedical_preference.json
✅ Configuration set!


## Load and Preprocess Preference Data

In [6]:
print("\n" + "="*80)
print("LOADING PREFERENCE DATA")
print("="*80)

def load_preference_data(
    file_path: str,
    num_samples: int = 1000,
    seed: int = 42
) -> Dataset:
    """
    Load and preprocess preference data from JSON file
    
    Expected format:
    {
        "prompt": "...",
        "chosen": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}],
        "rejected": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]
    }
    """

    print("Loading preference data...")

    # Load JSON
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Normalize structure
    if isinstance(data, dict):
        if "data" in data:
            data = data["data"]
        elif "rows" in data:
            data = data["rows"]

    print(f"Total available preference pairs: {len(data)}")

    # 🔹 Random sampling
    random.seed(seed)
    if len(data) > num_samples:
        data = random.sample(data, num_samples)
        print(f"Randomly sampled {num_samples} examples")
    else:
        print("Dataset smaller than requested sample size — using all data")

    processed_data = []

    for idx, example in enumerate(data):
        try:
            # Prompt
            prompt = example.get("prompt")
            if not prompt and "chosen" in example and isinstance(example["chosen"], list):
                prompt = example["chosen"][0].get("content", "")

            if not prompt:
                continue

            # Chosen
            if isinstance(example.get("chosen"), list):
                chosen = next(
                    (t["content"] for t in example["chosen"] if t.get("role") == "assistant"),
                    None
                )
            else:
                chosen = example.get("chosen")

            # Rejected
            if isinstance(example.get("rejected"), list):
                rejected = next(
                    (t["content"] for t in example["rejected"] if t.get("role") == "assistant"),
                    None
                )
            else:
                rejected = example.get("rejected")

            if chosen and rejected:
                processed_data.append({
                    "prompt": prompt,
                    "chosen": chosen,
                    "rejected": rejected
                })

        except Exception as e:
            print(f"⚠️ Skipped example {idx}: {e}")

    print(f"Successfully processed {len(processed_data)} examples")

    dataset = Dataset.from_pandas(pd.DataFrame(processed_data))

    # Preview
    print("\n📊 Sample:")
    print("Prompt:", dataset[0]["prompt"][:100], "...")
    print("Chosen:", dataset[0]["chosen"][:100], "...")
    print("Rejected:", dataset[0]["rejected"][:100], "...")

    return dataset


# Load the data
preference_dataset = load_preference_data(PREFERENCE_DATA_PATH)


LOADING PREFERENCE DATA
Loading preference data...
Total available preference pairs: 109353
Randomly sampled 1000 examples
Successfully processed 1000 examples

📊 Sample:
Prompt: Ive been recently having more nose bleeds lately. 4-5 times per day to be exact. And I have the wors ...
Chosen: 

I'm so sorry to hear that you're experiencing frequent nosebleeds and a persistent cough. While it ...
Rejected: I'm not a doctor, but I can provide you with some general information. Frequent nosebleeds accompani ...


In [7]:
# Split into train/eval (90/10)
split_dataset = preference_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print(f"\n✅ Dataset prepared!")
print(f"   Training samples: {len(train_dataset)}")
print(f"   Evaluation samples: {len(eval_dataset)}")


✅ Dataset prepared!
   Training samples: 900
   Evaluation samples: 100


## Format Data for DPO

In [8]:
print("\n" + "="*80)
print("FORMATTING DATA FOR DPO")
print("="*80)

def format_for_dpo(examples):
    """
    Format examples for DPO training
    DPO expects: prompt, chosen, rejected as formatted text
    """
    prompts = examples['prompt']
    chosen = examples['chosen']
    rejected = examples['rejected']
    
    # Format in Llama chat format
    def format_conversation(prompt, response):
        text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|>"
        text += f"<|start_header_id|>assistant<|end_header_id|>\n\n{response}<|eot_id|>"
        return text
    
    # Format chosen and rejected
    chosen_formatted = [format_conversation(p, c) for p, c in zip(prompts, chosen)]
    rejected_formatted = [format_conversation(p, r) for p, r in zip(prompts, rejected)]
    
    return {
        'prompt': prompts,
        'chosen': chosen_formatted,
        'rejected': rejected_formatted,
    }

# Apply formatting
print("Formatting training data...")
formatted_train = train_dataset.map(
    format_for_dpo,
    batched=True,
    remove_columns=train_dataset.column_names
)

print("Formatting evaluation data...")
formatted_eval = eval_dataset.map(
    format_for_dpo,
    batched=True,
    remove_columns=eval_dataset.column_names
)

print("✅ Data formatting complete!")
print(f"\nSample formatted data:")
print(f"Chosen (first 200 chars): {formatted_train[0]['chosen'][:200]}...")
print(f"Rejected (first 200 chars): {formatted_train[0]['rejected'][:200]}...")


FORMATTING DATA FOR DPO
Formatting training data...


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Formatting evaluation data...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Data formatting complete!

Sample formatted data:
Chosen (first 200 chars): <|begin_of_text|><|start_header_id|>user<|end_header_id|>

A 25 yr old female presented with malaise and generalized weakness since 6 months. Her appetite is reduced and she has giddiness and palpitat...
Rejected (first 200 chars): <|begin_of_text|><|start_header_id|>user<|end_header_id|>

A 25 yr old female presented with malaise and generalized weakness since 6 months. Her appetite is reduced and she has giddiness and palpitat...


## Load Tokenizer

In [9]:
print("\n" + "="*80)
print("LOADING TOKENIZER")
print("="*80)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded!")
print(f"   Vocab size: {len(tokenizer)}")
print(f"   PAD token: {tokenizer.pad_token}")
print(f"   EOS token: {tokenizer.eos_token}")


LOADING TOKENIZER


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

✅ Tokenizer loaded!
   Vocab size: 128256
   PAD token: <|eot_id|>
   EOS token: <|eot_id|>


## Load Model_SFT

In [10]:
print("\n" + "="*80)
print("LOADING MODEL_SFT")
print("="*80)

# Load base model
print(f"Loading base model: {BASE_MODEL}...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=BNB_CONFIG,
    device_map="auto",
    trust_remote_code=True,
)

print("✅ Base model loaded!")

# Load LoRA adapter (Model_SFT)
print(f"\nLoading Model_SFT from: {MODEL_SFT_PATH}...")
try:
    model = PeftModel.from_pretrained(base_model, MODEL_SFT_PATH)
    print("✅ Model_SFT (with LoRA) loaded!")
    
    # IMPORTANT: Enable gradient checkpointing for LoRA layers
    model.enable_input_require_grads()
    
    # Make LoRA parameters trainable
    for name, param in model.named_parameters():
        if 'lora' in name.lower():
            param.requires_grad = True
    
    print("✅ LoRA parameters unfrozen!")
    
except Exception as e:
    print(f"⚠️  Could not load LoRA adapter: {str(e)}")
    print("Continuing with base model (will add LoRA)")
    model = base_model
    
    # Prepare for k-bit training
    model = prepare_model_for_kbit_training(model)
    
    # Add LoRA
    lora_config = LoraConfig(**LORA_CONFIG)
    model = get_peft_model(model, lora_config)
    print("✅ LoRA added to base model!")

# Print model info
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 Model Statistics:")
print(f"   Trainable parameters: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")
print(f"   Total parameters: {total_params:,}")

# Verify trainable parameters
if trainable_params == 0:
    print("\n❌ ERROR: No trainable parameters!")
    print("This means the model won't train. Checking what went wrong...")
    
    # Debug: Print parameter names
    print("\nParameter status:")
    for name, param in model.named_parameters():
        if 'lora' in name.lower():
            print(f"   {name}: requires_grad={param.requires_grad}")
    
    raise ValueError("No trainable parameters found! Model cannot be trained.")
elif trainable_params < 1_000_000:
    print(f"\n⚠️  WARNING: Very few trainable parameters ({trainable_params:,})")
    print("Expected: 20-100 million for LoRA")
else:
    print(f"\n✅ Trainable parameters look good!")



LOADING MODEL_SFT
Loading base model: meta-llama/Llama-3.2-3B-Instruct...


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

✅ Base model loaded!

Loading Model_SFT from: /kaggle/input/ultramedical-model-sft-3b/other/default/1/Model_SFT_3B...
✅ Model_SFT (with LoRA) loaded!
✅ LoRA parameters unfrozen!

📊 Model Statistics:
   Trainable parameters: 9,175,040 (0.51%)
   Total parameters: 1,812,638,720

✅ Trainable parameters look good!


## Create DPO Trainer

In [11]:
print("\n" + "="*80)
print("CREATING DPO TRAINER")
print("="*80)

# DPO Configuration
dpo_config = DPOConfig(**DPO_CONFIG)

print("Configuration:")
print(f"   Epochs: {dpo_config.num_train_epochs}")
print(f"   Batch size: {dpo_config.per_device_train_batch_size}")
print(f"   Gradient accumulation: {dpo_config.gradient_accumulation_steps}")
print(f"   Learning rate: {dpo_config.learning_rate}")
print(f"   Beta (temperature): {dpo_config.beta}")

# Create DPO Trainer
print("\nInitializing DPO trainer...")
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # Will create frozen copy automatically
    args=dpo_config,
    train_dataset=formatted_train,
    eval_dataset=formatted_eval,
)

print("✅ DPO Trainer created successfully!")


CREATING DPO TRAINER
Configuration:
   Epochs: 1
   Batch size: 1
   Gradient accumulation: 16
   Learning rate: 5e-06
   Beta (temperature): 0.1

Initializing DPO trainer...


Extracting prompt in train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ DPO Trainer created successfully!


## Train Model D with DPO

In [12]:
print("\n" + "="*80)
print("TRAINING MODEL D - DPO (MANUAL LOOP)")
print("="*80)
print("🚀 Starting DPO training...")
print("Expected duration: 30-60 minutes on T4 GPU\n")

from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup
import copy

# ============================================================================
# Create Reference Model (Frozen Copy)
# ============================================================================
print("Creating frozen reference model...")
ref_model = copy.deepcopy(model).eval()
for param in ref_model.parameters():
    param.requires_grad = False
ref_model = ref_model.to('cpu')  # Keep on CPU to save GPU memory
print("✅ Reference model created on CPU")

# ============================================================================
# Setup Training
# ============================================================================
device = model.device if hasattr(model, 'device') else next(model.parameters()).device
print(f"Training device: {device}")

# Dataloader
train_dataloader = DataLoader(
    formatted_train,
    batch_size=DPO_CONFIG["per_device_train_batch_size"],
    shuffle=True,
    drop_last=True,
)

eval_dataloader = DataLoader(
    formatted_eval,
    batch_size=DPO_CONFIG["per_device_train_batch_size"],
    shuffle=False,
    drop_last=True,
) if len(formatted_eval) > 0 else None

# Optimizer
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=DPO_CONFIG["learning_rate"]
)

# Learning rate scheduler
num_training_steps = len(train_dataloader) * DPO_CONFIG["num_train_epochs"]
num_warmup_steps = int(num_training_steps * DPO_CONFIG["warmup_ratio"])

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# DPO hyperparameters
beta = DPO_CONFIG["beta"]
gradient_accumulation_steps = DPO_CONFIG["gradient_accumulation_steps"]

print(f"\n📊 Training Setup:")
print(f"   Total steps: {num_training_steps}")
print(f"   Warmup steps: {num_warmup_steps}")
print(f"   Gradient accumulation: {gradient_accumulation_steps}")
print(f"   Beta (temperature): {beta}")

# ============================================================================
# DPO Loss Function
# ============================================================================
def dpo_loss(policy_chosen_logps, policy_rejected_logps, 
             reference_chosen_logps, reference_rejected_logps, beta=0.1):
    """
    DPO loss calculation
    
    Loss = -log(σ(β * (log π_θ(y_w|x) - log π_θ(y_l|x) - log π_ref(y_w|x) + log π_ref(y_l|x))))
    
    Where:
    - π_θ is the policy model
    - π_ref is the reference model
    - y_w is the chosen (winning) response
    - y_l is the rejected (losing) response
    - β is the temperature parameter
    - σ is the sigmoid function
    """
    # Calculate log ratios
    policy_logratios = policy_chosen_logps - policy_rejected_logps
    reference_logratios = reference_chosen_logps - reference_rejected_logps
    
    # DPO loss
    logits = policy_logratios - reference_logratios
    losses = -torch.nn.functional.logsigmoid(beta * logits)
    
    # Calculate metrics
    chosen_rewards = beta * (policy_chosen_logps - reference_chosen_logps)
    rejected_rewards = beta * (policy_rejected_logps - reference_rejected_logps)
    
    return losses.mean(), chosen_rewards.mean(), rejected_rewards.mean()

def get_logprobs(model, input_ids, attention_mask):
    """Calculate log probabilities for sequences"""
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    
    # Shift logits and labels for next-token prediction
    logits = logits[:, :-1, :]
    labels = input_ids[:, 1:]
    attention_mask = attention_mask[:, 1:]
    
    # Get log probabilities
    logprobs = torch.nn.functional.log_softmax(logits, dim=-1)
    
    # Gather log probabilities for the actual tokens
    logprobs = torch.gather(logprobs, 2, labels.unsqueeze(-1)).squeeze(-1)
    
    # Mask out padding tokens
    logprobs = logprobs * attention_mask
    
    # Sum over sequence length
    return logprobs.sum(-1)

# ============================================================================
# Training Loop
# ============================================================================
model.train()
global_step = 0

for epoch in range(DPO_CONFIG["num_train_epochs"]):
    print(f"\n{'='*80}")
    print(f"EPOCH {epoch + 1}/{DPO_CONFIG['num_train_epochs']}")
    print(f"{'='*80}")
    
    epoch_loss = 0
    epoch_chosen_rewards = 0
    epoch_rejected_rewards = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}")
    
    for step, batch in enumerate(progress_bar):
        # Tokenize chosen responses
        chosen_inputs = tokenizer(
            batch['chosen'],
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=DPO_CONFIG["max_length"]
        )
        chosen_inputs = {k: v.to(device) for k, v in chosen_inputs.items()}
        
        # Tokenize rejected responses
        rejected_inputs = tokenizer(
            batch['rejected'],
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=DPO_CONFIG["max_length"]
        )
        rejected_inputs = {k: v.to(device) for k, v in rejected_inputs.items()}
        
        # Get policy model log probabilities
        policy_chosen_logps = get_logprobs(model, **chosen_inputs)
        policy_rejected_logps = get_logprobs(model, **rejected_inputs)
        
        # Get reference model log probabilities (on CPU)
        with torch.no_grad():
            # Move inputs to CPU temporarily
            chosen_inputs_cpu = {k: v.cpu() for k, v in chosen_inputs.items()}
            rejected_inputs_cpu = {k: v.cpu() for k, v in rejected_inputs.items()}
            
            reference_chosen_logps = get_logprobs(ref_model, **chosen_inputs_cpu).to(device)
            reference_rejected_logps = get_logprobs(ref_model, **rejected_inputs_cpu).to(device)
        
        # Calculate DPO loss
        loss, chosen_rewards, rejected_rewards = dpo_loss(
            policy_chosen_logps,
            policy_rejected_logps,
            reference_chosen_logps,
            reference_rejected_logps,
            beta=beta
        )
        
        # Scale loss for gradient accumulation
        loss = loss / gradient_accumulation_steps
        
        # Backward pass
        loss.backward()
        
        epoch_loss += loss.item()
        epoch_chosen_rewards += chosen_rewards.item()
        epoch_rejected_rewards += rejected_rewards.item()
        
        # Update weights
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': f'{loss.item() * gradient_accumulation_steps:.4f}',
            'reward_diff': f'{(chosen_rewards - rejected_rewards).item():.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
        
        # Logging
        if (step + 1) % DPO_CONFIG["logging_steps"] == 0:
            avg_loss = (epoch_loss * gradient_accumulation_steps) / (step + 1)
            avg_reward_diff = (epoch_chosen_rewards - epoch_rejected_rewards) / (step + 1)
            print(f"\n  Step {step + 1}: loss={avg_loss:.4f}, reward_diff={avg_reward_diff:.4f}")
        
        # Clear cache periodically
        if (step + 1) % 50 == 0:
            torch.cuda.empty_cache()
    
    # Epoch summary
    avg_loss = (epoch_loss * gradient_accumulation_steps) / len(train_dataloader)
    avg_chosen = epoch_chosen_rewards / len(train_dataloader)
    avg_rejected = epoch_rejected_rewards / len(train_dataloader)
    
    print(f"\n✅ Epoch {epoch + 1} Complete:")
    print(f"   Average Loss: {avg_loss:.4f}")
    print(f"   Chosen Rewards: {avg_chosen:.4f}")
    print(f"   Rejected Rewards: {avg_rejected:.4f}")
    print(f"   Reward Margin: {avg_chosen - avg_rejected:.4f}")
    
    # Evaluation
    if eval_dataloader and len(eval_dataloader) > 0:
        print(f"\n📊 Running evaluation...")
        model.eval()
        eval_loss = 0
        eval_chosen_rewards = 0
        eval_rejected_rewards = 0
        
        with torch.no_grad():
            for batch in tqdm(eval_dataloader, desc="Evaluating"):
                chosen_inputs = tokenizer(
                    batch['chosen'],
                    return_tensors='pt',
                    padding=True,
                    truncation=True,
                    max_length=DPO_CONFIG["max_length"]
                )
                chosen_inputs = {k: v.to(device) for k, v in chosen_inputs.items()}
                
                rejected_inputs = tokenizer(
                    batch['rejected'],
                    return_tensors='pt',
                    padding=True,
                    truncation=True,
                    max_length=DPO_CONFIG["max_length"]
                )
                rejected_inputs = {k: v.to(device) for k, v in rejected_inputs.items()}
                
                policy_chosen_logps = get_logprobs(model, **chosen_inputs)
                policy_rejected_logps = get_logprobs(model, **rejected_inputs)
                
                chosen_inputs_cpu = {k: v.cpu() for k, v in chosen_inputs.items()}
                rejected_inputs_cpu = {k: v.cpu() for k, v in rejected_inputs.items()}
                
                reference_chosen_logps = get_logprobs(ref_model, **chosen_inputs_cpu).to(device)
                reference_rejected_logps = get_logprobs(ref_model, **rejected_inputs_cpu).to(device)
                
                loss, chosen_rewards, rejected_rewards = dpo_loss(
                    policy_chosen_logps,
                    policy_rejected_logps,
                    reference_chosen_logps,
                    reference_rejected_logps,
                    beta=beta
                )
                
                eval_loss += loss.item()
                eval_chosen_rewards += chosen_rewards.item()
                eval_rejected_rewards += rejected_rewards.item()
        
        avg_eval_loss = eval_loss / len(eval_dataloader)
        avg_eval_chosen = eval_chosen_rewards / len(eval_dataloader)
        avg_eval_rejected = eval_rejected_rewards / len(eval_dataloader)
        
        print(f"✅ Evaluation Results:")
        print(f"   Loss: {avg_eval_loss:.4f}")
        print(f"   Reward Margin: {avg_eval_chosen - avg_eval_rejected:.4f}")
        
        model.train()

print("\n" + "="*80)
print("✅ DPO TRAINING COMPLETE!")
print("="*80)


TRAINING MODEL D - DPO (MANUAL LOOP)
🚀 Starting DPO training...
Expected duration: 30-60 minutes on T4 GPU

Creating frozen reference model...
✅ Reference model created on CPU
Training device: cuda:0

📊 Training Setup:
   Total steps: 900
   Warmup steps: 90
   Gradient accumulation: 16
   Beta (temperature): 0.1

EPOCH 1/1


Training Epoch 1:   0%|          | 1/900 [04:04<60:55:59, 244.00s/it, loss=0.6934, reward_diff=0.0000, lr=0.00e+00]


KeyboardInterrupt: 

## Save Model D

In [ ]:
print("\n" + "="*80)
print("SAVING MODEL D")
print("="*80)

# Save model
print(f"Saving to: {MODEL_D_PATH}...")
dpo_trainer.save_model(MODEL_D_PATH)
tokenizer.save_pretrained(MODEL_D_PATH)

print(f"✅ Model D saved successfully!")

# Zip the model for easy download
import shutil
print("\nCreating zip archive...")
shutil.make_archive(MODEL_D_PATH, 'zip', MODEL_D_PATH)
print(f"✅ Model zipped: {MODEL_D_PATH}.zip")

## Test Model D

In [ ]:
print("\n" + "="*80)
print("TESTING MODEL D")
print("="*80)

# Set model to eval mode
model.eval()

# Test prompt
test_prompt = "A 55-year-old man presents with chest pain and shortness of breath. What are the key differential diagnoses to consider?"

print(f"Test Prompt: {test_prompt}\n")

# Format prompt
formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{test_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

# Tokenize
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

# Generate
print("Generating response...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# Decode
full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract assistant's response
if "assistant<|end_header_id|>" in full_response:
    response = full_response.split("assistant<|end_header_id|>")[-1].strip()
else:
    response = full_response

print("="*80)
print("MODEL D RESPONSE:")
print("="*80)
print(response)
print("="*80)